# Importer les bibliothèques nécessaires

In [118]:
import pandas as pd
import ast 
import numpy as np
import csv

# Importer les données de Doctolib

In [119]:
data = pd.read_csv('data/extracted_data_final_0_1000.csv')

In [120]:
# Fonction pour réorganiser les profils cards
def reorder_cards(profile_cards):
    try:
        cards_dict = ast.literal_eval(profile_cards)
        
        reordered_cards = {
            'card_0': np.nan,
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

        def split_content(value):
            value = value.replace('\\n', '\n').replace('\\', '\n')
            return value.split('\n')

        for key, value in cards_dict.items():
            #print(key,value,'s')
            if value.startswith("Carte et informations d'accès"):
                #print('1')
                reordered_cards['card_1'] = split_content(value)
            elif value.startswith("Présentation"):
                #print('2')
                reordered_cards['card_2'] = split_content(value)
            elif value.startswith("Horaires et coordonnées"):
                #print('3')
                reordered_cards['card_3'] = split_content(value)
            elif value.startswith("Tarifs et remboursement") or value.startswith("Lieu"):
                #print('i')
                reordered_cards['card_0'] = split_content(value)
            elif value.startswith("Tarifs"):
                #print('4')
                reordered_cards['card_4'] = split_content(value)
            elif value.startswith("Informations légales"):
                #print('5')
                reordered_cards['card_5'] = split_content(value)
            else :
                if isinstance(value,list):
                    #print('i1')
                    reordered_cards['card_0'] = split_content(value)
        
        return reordered_cards
    except:
        return {
            'card_0': np.nan,
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

reordered_cards = data['Profile Cards'].apply(reorder_cards)

reordered_cards_df = pd.json_normalize(reordered_cards)

data_combined = pd.concat([data, reordered_cards_df], axis=1)

data_combined.drop(columns = 'Profile Cards', inplace=True)
print(data_combined[['Doctor Name', 'card_0', 'card_1', 'card_2', 'card_3', 'card_4', 'card_5']].tail(9))


                   Doctor Name  \
991        Dr Mathieu SABATIER   
992           Dr Eric LE FLOCH   
993    Agnès Montagnon -Thomas   
994     Dr Stella Danan-Hadida   
995  Dr Philippe Lazare SELLAM   
996          Dr Michèle Maizil   
997           Dr Thierry MEYER   
998          Dr Marc Stromboni   
999               Fabien GUIDE   

                                                card_0  \
991                                                NaN   
992  [Lieu 1, Lieu 2, Lieu 3, Radio Faidherbe, Radi...   
993  [Tarifs et remboursement, , , Tiers payant : S...   
994  [Tarifs et remboursement, , , , Conventionné s...   
995  [Lieu 1, Lieu 2, Cabinet du Docteur Philippe L...   
996                                                NaN   
997  [Tarifs et remboursement, , , , Conventionné, ...   
998  [Tarifs et remboursement, , Conventionné secte...   
999  [Tarifs et remboursement, , , , Conventionné, ...   

                                                card_1  \
991  [Carte et infor

In [121]:
#Pour rendre la localisation plus claire
def extract_lat_lng(map_location):
    try:
        location_dict = ast.literal_eval(map_location)
        return (location_dict['lat'], location_dict['lng'])
    except:
        return (None, None)

data_combined['Latitude'], data_combined['Longitude'] = zip(*data_combined['Map Location'].apply(extract_lat_lng))

data_combined.drop(columns = 'Map Location', inplace=True)


In [122]:
# Fonction pour traiter la colonne 'Skills'
def process_skills(skills):
    if skills == 'Skills missing':
        return np.nan
    else:
        return skills.split('\n')

data_combined['Skills Processed'] = data_combined['Skills'].apply(process_skills)

data_combined.drop(columns = 'Skills', inplace=True)

In [123]:
data_combined['Specialty'].value_counts()

Specialty
Chirurgien-dentiste                                                 152
Médecin généraliste                                                 104
Ophtalmologue                                                        75
Cabinet médical                                                      56
Spécialiste en chirurgie plastique reconstructrice et esthétique     40
                                                                   ... 
Chirurgien oral                                                       1
Acupuncteur                                                           1
Addictologue                                                          1
Chirurgien cancérologue                                               1
Diététicien                                                           1
Name: count, Length: 78, dtype: int64

In [124]:
data_combined['RPPS'] = data_combined['card_5'].apply(lambda x: x[2] if isinstance(x, list) and len(x) > 2 else np.nan)
data_combined.drop(columns = 'card_5', inplace = True)

In [125]:
data_combined['Nb Skills'] = data_combined['Skills Processed'].apply(lambda x: len(x)-1 if isinstance(x, list) else np.nan)

In [126]:
def paiement(card0):
    if isinstance(card0, list):
        if 'Moyens de paiement' in card0:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['moyens_paiement'] = data_combined['card_0'].apply(paiement)


def vitale(card0):
    if isinstance(card0, list):
        if 'Carte Vitale acceptée' in card0:
            return 1
        elif 'Carte Vitale non acceptée' in card0:
            return 0
        else:
            return np.nan
    else:
        return np.nan
data_combined['carte_vitale'] = data_combined['card_0'].apply(paiement)


def secteur(card0):
    if isinstance(card0, list):
        a=np.nan
        for x in card0:
            if 'Conventionné secteur 2' in x:
                a=2
            elif 'Conventionné secteur 1' in x:
                a=1
            elif 'Conventionné secteur 3' in x:
                a=3
            elif 'Conventionné' in x:
                a=0
            else:
                a=a
        return a
    else:
        return np.nan
data_combined['secteur'] = data_combined['card_0'].apply(secteur)

In [127]:
data_combined['Présentation']=''
data_combined['Langues parlées']=''
data_combined['Diplômes nationaux et universitaires']=''
data_combined['Autres formations']=''
data_combined['Expériences']=''
data_combined['Travaux et publications']=''
data_combined['Prix et distinctions']=''
data_combined['site']=0

liste_mots_clefs = ['Présentation','Langues parlées','Diplômes nationaux et universitaires','Formations','Autres formations','Expériences', 'Site web', 'Travaux et publications', 'Prix et distinctions']

for i in range(data_combined.shape[0]):
    
    if isinstance(data_combined['card_2'].iloc[i],list) : 
        liste_card2 = data_combined['card_2'].iloc[i][1:]
        sous_liste = ''
        label = liste_mots_clefs[0]
        for x in liste_card2 :
            if x in liste_mots_clefs :
                data_combined.loc[i, label] = sous_liste
                label = x
                sous_liste = ''
            elif x=='Voir le site':
                data_combined.loc[i, 'site'] = 1
            elif x=='▾ Voir plus':
                None
            else :
                sous_liste = sous_liste+(x)

In [128]:
data_combined['Diplômes nationaux et universitaires b']=data_combined['Diplômes nationaux et universitaires'].apply(lambda x: 1 if len(x)>0 else 0)

In [129]:
data_combined['Nb caractères présentation'] = data_combined['Présentation'].apply(lambda x: len(x))

In [130]:
data_combined['Autres formations b']=data_combined['Autres formations'].apply(lambda x: 1 if len(x)>0 else 0)

In [131]:
data_combined['Nb langues']=0

for i in range(data_combined.shape[0]):
    if isinstance(data_combined['Langues parlées'].iloc[i],str) : 
        liste_langue = (data_combined['Langues parlées'].iloc[i]).split()
        count = 0
        for x in liste_langue :
            
            if  x != 'et'  :
                count +=1
        data_combined.loc[i,'Nb langues']=count

In [132]:
data_combined['Expériences b']=data_combined['Expériences'].apply(lambda x: 1 if len(x)>0 else 0)

In [133]:
liste1 = ["Carte et informations d'accès",'Moyens de transport','Parking public','Informations pratiques',"J'autorise le traitement d'informations (dont mon adresse IP) et leur transfert hors UE par Google Maps (USA) afin d’afficher la carte.En savoir plus", 'sur la collecte et le traitement des données par Google','AFFICHER LA CARTE']
def carac1(card1):
    nb=0
    for phrase in card1:
        if  phrase not in liste1:
            nb += len(phrase)
    return nb
data_combined['Carac_card_1'] = data_combined['card_1'].apply(lambda x: carac1(x)if isinstance(x, list) else np.nan)

In [134]:
def transport(card1):
    if isinstance(card1, list):
        if 'Moyens de transport' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Transport'] = data_combined['card_1'].apply(transport)

In [135]:
def parking(card1):
    if isinstance(card1, list):
        if 'Parking public' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Parking'] = data_combined['card_1'].apply(parking)

In [136]:
def infos(card1):
    if isinstance(card1, list):
        if 'Informations pratiques' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Infos'] = data_combined['card_1'].apply(infos)

In [137]:
def coordonnées(card3):
    if isinstance(card3, list):
        if 'Coordonnées' in card3:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Coordonnées'] = data_combined['card_3'].apply(coordonnées) 

In [138]:
def sansRDV(card3):
    if isinstance(card3, list):
        if 'Consultations sans rendez-vous' in card3:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Sans RDV'] = data_combined['card_3'].apply(sansRDV)

In [139]:
data_combined.drop(columns=['URL','card_1','card_2'],inplace=True)
data_combined.head()

,Doctor Name,Specialty,card_0,card_3,card_4,Latitude,Longitude,Skills Processed,RPPS,Nb Skills,...,Nb caractères présentation,Autres formations b,Nb langues,Expériences b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV
0,Dr Marian AGACHI,Neurochirurgien,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,48.878328,2.431034,"[Expertises et actes, Neurochirurgie du rachis]",10001649291,1.0,...,748,0,3,0,154.0,1,0,1,1,0
1,Dr Anne Vaillant Moga,Médecin généraliste,"[Lieu 1, Lieu 2, Dr Anne Moga (Paris), 76 aven...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation préalable de médecine es...",48.869218,2.285468,"[Expertises et actes, Acné, Allergie, Apnée du...",10000378991,10.0,...,2538,1,2,1,180.0,1,0,1,0,0
2,Dr Gabriel Ohana,Chirurgien-dentiste,"[Tarifs et remboursement, , Carte Vitale accep...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation dentaire, 23 €, Traiteme...",48.894148,2.250698,"[Expertises et actes, Chirurgie buccale, Proth...",10003639373,3.0,...,481,1,1,0,240.0,1,0,1,1,0
3,Dr Thomas BAMBERGER,Dermatologue et vénérologue,"[Tarifs et remboursement, , , Conventionné sec...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation, 31,50 € à 56,50 €, Cont...",48.815433,2.343017,NaN,10003719571,NaN,...,407,0,1,1,155.0,1,0,1,1,0
4,Dr Layla Chatila,Chirurgien-dentiste,"[Tarifs et remboursement, , , , Conventionné, ...","[Horaires et coordonnées, Coordonnées, 01 45 2...","[Tarifs, Consultation dentaire, 23 €, Ces hono...",48.858804,2.274392,NaN,10005133003,NaN,...,246,0,1,0,202.0,1,1,1,1,0


# Problème des nan

In [140]:
indices_nan = data_combined[data_combined['RPPS'].isna()].index.tolist()
print(indices_nan, len(indices_nan))

[8, 23, 27, 73, 74, 94, 112, 116, 117, 118, 121, 122, 123, 124, 127, 130, 131, 132, 135, 136, 138, 139, 140, 141, 142, 143, 145, 148, 150, 151, 152, 153, 156, 157, 158, 160, 161, 163, 166, 167, 169, 170, 172, 173, 175, 176, 177, 178, 179, 180, 182, 183, 184, 186, 187, 188, 190, 191, 192, 194, 196, 197, 198, 201, 202, 204, 209, 213, 214, 215, 216, 219, 220, 221, 222, 223, 225, 226, 227, 229, 230, 231, 233, 234, 235, 236, 238, 239, 240, 241, 245, 246, 247, 249, 250, 252, 254, 256, 259, 260, 262, 263, 264, 265, 267, 268, 270, 271, 272, 276, 277, 278, 279, 280, 281, 282, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 312, 315, 316, 317, 318, 319, 320, 321, 323, 324, 325, 326, 327, 328, 329, 331, 335, 337, 339, 344, 346, 347, 348, 349, 350, 351, 352, 353, 354, 356, 358, 359, 360, 365, 367, 368, 371, 372, 373, 374, 376, 378, 379, 380, 381, 382, 383, 384, 387, 388, 390, 392, 393, 395, 396, 397, 398, 399, 400, 4

In [176]:
data['Profile Cards'].iloc[999]

'{\'card_0\': \'Tarifs et remboursement\\n\\n\\n\\nConventionné\\nCarte Vitale acceptée\\nTiers payant : Sécurité sociale\\nVoir les tarifs\\nMoyens de paiement\\nChèques, espèces et carte bancaire\\nExpertises et actes\\nBilan podologique\\nIonophorèse\\nK-Tape\\nKinesio Taping\\nOrthonyxie\\nOrthoplastie\\nOrthèse plantaire\\nPodologie\\nPédicurie\\nSemelles orthopédiques\\nVerrue\', \'card_1\': "Carte et informations d\'accès\\n9 Avenue Ampère, 77420 Champs-sur-Marne\\nMoyens de transport\\nBus - Ampere (lignes 213 et 312)\\nBus - Noisy-Champs Rer - Descartes (ligne 212)\\nInformations pratiques\\n1er étage avec ascenseur\\nEntrée accessible\\nJ\'autorise le traitement d\'informations (dont mon adresse IP) et leur transfert hors UE par Google Maps (USA) afin d’afficher la carte.En savoir plus\\nsur la collecte et le traitement des données par Google\\nAFFICHER LA CARTE", \'card_2\': "Présentation\\nDepuis janvier 2010, je vous reçois dans mon cabinet à Champs-sur-Marne à deux pas de

In [142]:
data_combined_sans_nan = data_combined[data_combined['RPPS'].notna()]
data_combined_sans_nan

,Doctor Name,Specialty,card_0,card_3,card_4,Latitude,Longitude,Skills Processed,RPPS,Nb Skills,...,Nb caractères présentation,Autres formations b,Nb langues,Expériences b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV
0,Dr Marian AGACHI,Neurochirurgien,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,48.878328,2.431034,"[Expertises et actes, Neurochirurgie du rachis]",10001649291,1.0,...,748,0,3,0,154.0,1,0,1,1,0
1,Dr Anne Vaillant Moga,Médecin généraliste,"[Lieu 1, Lieu 2, Dr Anne Moga (Paris), 76 aven...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation préalable de médecine es...",48.869218,2.285468,"[Expertises et actes, Acné, Allergie, Apnée du...",10000378991,10.0,...,2538,1,2,1,180.0,1,0,1,0,0
2,Dr Gabriel Ohana,Chirurgien-dentiste,"[Tarifs et remboursement, , Carte Vitale accep...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation dentaire, 23 €, Traiteme...",48.894148,2.250698,"[Expertises et actes, Chirurgie buccale, Proth...",10003639373,3.0,...,481,1,1,0,240.0,1,0,1,1,0
3,Dr Thomas BAMBERGER,Dermatologue et vénérologue,"[Tarifs et remboursement, , , Conventionné sec...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation, 31,50 € à 56,50 €, Cont...",48.815433,2.343017,NaN,10003719571,NaN,...,407,0,1,1,155.0,1,0,1,1,0
4,Dr Layla Chatila,Chirurgien-dentiste,"[Tarifs et remboursement, , , , Conventionné, ...","[Horaires et coordonnées, Coordonnées, 01 45 2...","[Tarifs, Consultation dentaire, 23 €, Ces hono...",48.858804,2.274392,NaN,10005133003,NaN,...,246,0,1,0,202.0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,Dr Danielle Taconet,Ophtalmologue,"[Tarifs et remboursement, , , Conventionné sec...","[Horaires et coordonnées, Contact d'urgence, E...",NaN,49.060704,2.184674,"[Expertises et actes, Champ visuel, Laser opht...",10003749230,4.0,...,7800,0,2,0,231.0,1,0,1,1,0
994,Dr Stella Danan-Hadida,Gynécologue médicale et obstétrique,"[Tarifs et remboursement, , , , Conventionné s...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation vidéo, 58 €, Consultatio...",48.871909,2.346805,"[Expertises et actes, Gynécologie obstétrique,...",10000292291,7.0,...,626,0,1,0,249.0,1,1,1,1,0
995,Dr Philippe Lazare SELLAM,Spécialiste en chirurgie plastique reconstruct...,"[Lieu 1, Lieu 2, Cabinet du Docteur Philippe L...","[Horaires et coordonnées, Contact d'urgence, E...","[Tarifs, Consultation, 23 € à 120 €, Exérèse d...",48.868676,2.325944,NaN,10000558964,NaN,...,535,1,1,0,271.0,1,1,1,1,0
996,Dr Michèle Maizil,Chirurgien-dentiste,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,48.834970,2.246637,NaN,10000202985,NaN,...,371,0,0,0,376.0,1,1,1,1,0


# Ouverture base de données activités

In [202]:
annuaire = pd.read_csv('data/PS_LibreAcces_Personne_activite_small.csv')
annuaire.head()

/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_81949/1646588620.py:1: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  annuaire = pd.read_csv('data/PS_LibreAcces_Personne_activite_small.csv')


,typedidentifiantpp,identifiantpp,codecivilitédexercice,nomdexercice,prénomdexercice,codeprofession,libelléprofession,codecatégorieprofessionnelle,libellécatégorieprofessionnelle,codemodeexercice,...,codepayscoordstructure,libellépayscoordstructure,codedépartementstructure,libellédépartementstructure,codesecteurdactivité,libellésecteurdactivité,coderôle,libellérôle,codegenreactivité,libellégenreactivité
0,0,010001410,NaN,MANDRAY,Pascal,71,Ostéopathe,C,Civil,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,010003887,NaN,CATHERINE BONNICI,Julie,71,Ostéopathe,C,Civil,S,...,NaN,NaN,NaN,NaN,SA01,Etablissement Public de santé,NaN,NaN,NaN,NaN
2,0,010005478,NaN,REMY,Celia,71,Ostéopathe,C,Civil,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,010005684,NaN,VELON,Amelie,71,Ostéopathe,C,Civil,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,019111228,NaN,BOYER,MARILYS,91,Orthophoniste,C,Civil,L,...,NaN,NaN,NaN,NaN,SA07,Cabinet individuel,FON-01,Titulaire de cabinet,NaN,NaN


In [204]:
annuaire['libellépayscoordstructure'].value_counts()

libellépayscoordstructure
France       628198
Suisse          133
Espagne           5
Canada            4
Monaco            3
Belgique          3
Allemagne         1
Guyana            1
Name: count, dtype: int64

In [182]:
print(annuaire[annuaire['libellépayscoordstructure']=='France']['codepostalcoordstructure'].isnull().sum(),annuaire[annuaire['libellépayscoordstructure']=='France'].shape)

0 (628198, 33)


In [186]:
print(annuaire[annuaire['libellépayscoordstructure']!='France']['codepostalcoordstructure'].isnull().sum(),annuaire[annuaire['libellépayscoordstructure']!='France'].shape)

582188 (1428603, 33)


In [206]:
c=0
for metier in annuaire['libelléprofession'].unique() :
    print(metier,annuaire[annuaire['libelléprofession']==metier].shape,annuaire[annuaire['libelléprofession']==metier]['codepostalcoordstructure'].isnull().sum())
    c+=annuaire[annuaire['libelléprofession']==metier]['codepostalcoordstructure'].isnull().sum()
adre = annuaire.shape[0]-c
print(adre,adre/annuaire.shape[0],adre)

Ostéopathe (43601, 33) 42643
Orthophoniste (33682, 33) 4029
Psychothérapeute (27878, 33) 16769
Orthoptiste (9223, 33) 3116
Chiropracteur (2180, 33) 2167
Sage-Femme (38949, 33) 9874
Médecin (508186, 33) 177860
Chirurgien-Dentiste (71317, 33) 13986
Pharmacien (84931, 33) 10147
Orthopédiste-Orthésiste (3032, 33) 1551
Infirmier (613001, 33) 142472
Masseur-Kinésithérapeute (135710, 33) 14902
Pédicure-Podologue (17566, 33) 800
Assistant social (48737, 33) 26353
Assistant dentaire (22051, 33) 15569
Ergothérapeute (21669, 33) 3444
Psychomotricien (24514, 33) 3505
Manipulateur ERM (42939, 33) 10412
Psychologue (126837, 33) 37441
Diététicien (27409, 33) 9374
Technicien de Laboratoire (58927, 33) 8703
Physicien médical (516, 33) 17
Opticien-Lunetier (54661, 33) 22980
Audio-Prothésiste (9102, 33) 2380
Orthoprothésiste (2037, 33) 794
Podo-Orthésiste (932, 33) 419
Epithésiste (104, 33) 60
Acteur du système de santé caractérisé par un rôle (27031, 33) 392
Oculariste (79, 33) 29
1474613 0.716944906191

In [144]:
print(annuaire.shape[0],annuaire['codepostalcoordstructure'].isnull().sum())

2056801 582188


In [145]:
liste_med = annuaire.merge(data_combined,how='outer',left_on='identifiantpp',right_on='RPPS')

In [146]:
liste_med_2 = annuaire.merge(data_combined_sans_nan,how='outer',left_on='identifiantpp',right_on='RPPS')

In [147]:
print(liste_med_2.shape[0]-annuaire.shape[0], data_combined_sans_nan.shape[0],annuaire.shape[0])

3 367 2056801


In [148]:
liste_med_2_sans_nan = liste_med_2[liste_med_2['RPPS'].notna()]
print(liste_med_2_sans_nan.shape[0],liste_med_2_sans_nan['codepostalcoordstructure'].isnull().sum())

720 48


In [149]:
liste_med_2_sans_nan['RPPS'].value_counts()

RPPS
10004037247    9
10004402102    8
10002296464    8
10002292505    8
10005179634    7
              ..
10001625564    1
10001627800    1
10001645018    1
10002239217    1
950002402      1
Name: count, Length: 367, dtype: int64

In [150]:
regroupe = liste_med_2_sans_nan[liste_med_2_sans_nan['RPPS']=='10004037247']
regroupe.describe()

,typedidentifiantpp,codeprofession,codepayscoordstructure,codedépartementstructure,libellédépartementstructure,Latitude,Longitude,Nb Skills,moyens_paiement,carte_vitale,...,Nb caractères présentation,Autres formations b,Nb langues,Expériences b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV
count,9.0,9.0,7.0,0.0,0.0,9.000000,9.000000,9.0,9.0,9.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0
mean,8.0,10.0,99000.0,NaN,NaN,50.648965,3.085347,10.0,0.0,0.0,...,623.0,0.0,0.0,0.0,274.0,1.0,1.0,1.0,1.0,0.0
std,0.0,0.0,0.0,NaN,NaN,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,8.0,10.0,99000.0,NaN,NaN,50.648965,3.085347,10.0,0.0,0.0,...,623.0,0.0,0.0,0.0,274.0,1.0,1.0,1.0,1.0,0.0
25%,8.0,10.0,99000.0,NaN,NaN,50.648965,3.085347,10.0,0.0,0.0,...,623.0,0.0,0.0,0.0,274.0,1.0,1.0,1.0,1.0,0.0
50%,8.0,10.0,99000.0,NaN,NaN,50.648965,3.085347,10.0,0.0,0.0,...,623.0,0.0,0.0,0.0,274.0,1.0,1.0,1.0,1.0,0.0
75%,8.0,10.0,99000.0,NaN,NaN,50.648965,3.085347,10.0,0.0,0.0,...,623.0,0.0,0.0,0.0,274.0,1.0,1.0,1.0,1.0,0.0
max,8.0,10.0,99000.0,NaN,NaN,50.648965,3.085347,10.0,0.0,0.0,...,623.0,0.0,0.0,0.0,274.0,1.0,1.0,1.0,1.0,0.0


In [151]:
df_sans_nan_dup = liste_med_2_sans_nan.drop_duplicates(subset=['RPPS'])
print(df_sans_nan_dup.shape[0],df_sans_nan_dup['codepostalcoordstructure'].isnull().sum())


367 34


In [189]:
df_sans_nan_dup['codepostalcoordstructure'].describe()

count       333.0
unique      144.0
top       92100.0
freq         27.0
Name: codepostalcoordstructure, dtype: float64

In [190]:
indices_nan = df_sans_nan_dup[df_sans_nan_dup['codepostalcoordstructure'].isna()].index.tolist()
print(indices_nan, len(indices_nan))

[32959, 34236, 41858, 43189, 55718, 73734, 74292, 91405, 118393, 119349, 235459, 300663, 308350, 335858, 336641, 342278, 382629, 423857, 430441, 929088, 1948251, 2023935, 2023938, 2023978, 2030332, 2034069, 2042384, 2043380, 2046240, 2046246, 2046250, 2046310, 2046399, 2052848] 34


In [193]:
df_sans_nan_dup

,typedidentifiantpp,identifiantpp,codecivilitédexercice,nomdexercice,prénomdexercice,codeprofession,libelléprofession,codecatégorieprofessionnelle,libellécatégorieprofessionnelle,codemodeexercice,...,Nb caractères présentation,Autres formations b,Nb langues,Expériences b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV
5380,0.0,069204550,NaN,LAZZARESCHI,JULIE,92.0,Orthoptiste,C,Civil,L,...,302.0,0.0,1.0,0.0,215.0,1.0,1.0,1.0,1.0,0.0
7971,8.0,10000030766,DR,NALLET,EMMANUEL,10.0,Médecin,C,Civil,L,...,168.0,0.0,2.0,0.0,134.0,0.0,0.0,0.0,1.0,0.0
9397,8.0,10000058080,DR,ARNAUD,LOUIS,40.0,Chirurgien-Dentiste,C,Civil,L,...,307.0,0.0,2.0,0.0,233.0,1.0,1.0,1.0,1.0,0.0
9984,8.0,10000069749,DR,SALA,BERNARD,40.0,Chirurgien-Dentiste,C,Civil,L,...,586.0,0.0,2.0,0.0,286.0,1.0,1.0,1.0,0.0,0.0
10300,8.0,10000076546,DR,COHEN,THIERRY,40.0,Chirurgien-Dentiste,C,Civil,L,...,262.0,0.0,1.0,0.0,300.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046246,0.0,920003654,NaN,COURTHEOUX,Quentin,73.0,Chiropracteur,C,Civil,L,...,896.0,1.0,2.0,0.0,252.0,1.0,1.0,1.0,1.0,0.0
2046250,0.0,920003720,NaN,POINCLOUX,Margaux,71.0,Ostéopathe,C,Civil,S,...,533.0,1.0,1.0,1.0,190.0,1.0,0.0,1.0,0.0,0.0
2046310,0.0,920004546,NaN,PARDO,Johanna,71.0,Ostéopathe,C,Civil,L,...,570.0,0.0,1.0,0.0,230.0,1.0,1.0,1.0,1.0,0.0
2046399,0.0,920005824,NaN,KEFFI,Sarah,71.0,Ostéopathe,C,Civil,L,...,1797.0,0.0,2.0,0.0,290.0,1.0,1.0,1.0,1.0,0.0


In [152]:
df_sans_nan_dup_dep = df_sans_nan_dup[df_sans_nan_dup['codepostalcoordstructure'].notna()]

In [153]:
def departement(codepostal):
    code = int(codepostal)/1000
    if int(code) == 97 :
        return(str(int(codepostal/100)))
    if int(code) < 10 :
        return('0'+str(int(code)))
    return (str(int(code)))

In [162]:
df_sans_nan_dup_dep.loc[:, 'code dépar'] = df_sans_nan_dup_dep['codepostalcoordstructure'].apply(departement)
df_sans_nan_dup_dep['code dépar']

5380        06
7971       971
9397        92
9984        75
10300       75
          ... 
981728      06
982575      78
988570      75
997689      92
2035980     78
Name: code dépar, Length: 333, dtype: object

In [155]:
df_sans_nan_dup_dep['Specialty'].value_counts()

Specialty
Chirurgien-dentiste                                                 66
Médecin généraliste                                                 37
Ophtalmologue                                                       29
Gynécologue obstétricienne                                          14
Pédicure-podologue                                                  14
Spécialiste en chirurgie plastique reconstructrice et esthétique    13
Dermatologue et vénérologue                                         13
Gynécologue obstétricien                                            12
Masseur-kinésithérapeute                                            12
Chirurgien orthopédiste et traumatologue                             9
Pédiatre                                                             8
Cardiologue                                                          8
Chirurgien urologue                                                  8
Gastro-entérologue et hépatologue                                  

In [156]:
df_sans_nan_dup_dep.shape

(333, 68)

Ouverture BAN

In [157]:
ban = pd.read_csv('data/adresses-france.csv')
ban.head()

/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_81949/259408599.py:1: DtypeWarning: Columns (5,8,9,15,16,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  ban = pd.read_csv('data/adresses-france.csv')


,id,id_fantoir,numero,rep,nom_voie,code_postal,code_insee,nom_commune,code_insee_ancienne_commune,nom_ancienne_commune,...,type_position,alias,nom_ld,libelle_acheminement,nom_afnor,source_position,source_nom_voie,certification_commune,cad_parcelles,Unnamed: 23
0,01001_4b50r5_00630,NaN,630,NaN,la Chèvre,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,segment,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,LA CHEVRE,inconnue,inconnue,0,NaN,NaN
1,01001_g0ru02_00108,NaN,108,NaN,Clemencia,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,parcelle,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,CLEMENCIA,cadastre,cadastre,0,01001000ZL0263,NaN
2,01001_ngzlqw_00009,NaN,9,NaN,Imp des Epis,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,NaN,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,arcep,arcep,0,NaN,NaN
3,01001_ngzlqw_00023,NaN,23,NaN,Imp des Epis,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,NaN,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,arcep,arcep,0,NaN,NaN
4,01001_ngzlqw_00026,NaN,26,NaN,Imp des Epis,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,NaN,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,arcep,arcep,0,NaN,NaN


Ouverture concurrence d'après INSEE

In [165]:
concu = pd.read_excel('data/TCRD_068.xlsx')
concu.rename(columns={'Professionnels de santé au 1ᵉʳ janvier 2023 : comparaisons départementales' : 'Code Département','Unnamed: 1' : 'Nom Département','Unnamed: 2':'Nombre de médecins','Unnamed: 3':'Nombre de médecins pour 100 000 habitants','Unnamed: 4':'généralistes','Unnamed: 5':'spécialistes','Unnamed: 6':'dentistes','Unnamed: 7':'Pharm.'}, inplace=True)
concu.drop(labels = [0,1,2,3,108],axis=0, inplace=True)
concu

,Code Département,Nom Département,Nombre de médecins,Nombre de médecins pour 100 000 habitants,généralistes,spécialistes,dentistes,Pharm.
4,01,Ain,1162,174,99,75,53,78
5,02,Aisne,1107,211,99,111,45,91
6,03,Allier,917,275,134,141,51,111
7,04,Alpes-de-Haute-Provence,483,291,165,125,54,103
8,05,Hautes-Alpes,705,503,291,213,72,135
...,...,...,...,...,...,...,...,...
103,972,Martinique,1171,332,152,180,68,99
104,973,Guyane,717,242,123,120,30,46
105,974,La Réunion,3170,364,176,189,67,94
106,976,Mayotte,265,89,49,39,9,31


In [167]:
concu.head()

,Code Département,Nom Département,Nombre de médecins,Nombre de médecins pour 100 000 habitants,généralistes,spécialistes,dentistes,Pharm.
4,01,Ain,1162,174,99,75,53,78
5,02,Aisne,1107,211,99,111,45,91
6,03,Allier,917,275,134,141,51,111
7,04,Alpes-de-Haute-Provence,483,291,165,125,54,103
8,05,Hautes-Alpes,705,503,291,213,72,135


In [168]:
data_avec_concu = df_sans_nan_dup_dep.merge(concu, left_on='code dépar', right_on='Code Département',how='inner')

In [169]:
data_avec_concu

,typedidentifiantpp,identifiantpp,codecivilitédexercice,nomdexercice,prénomdexercice,codeprofession,libelléprofession,codecatégorieprofessionnelle,libellécatégorieprofessionnelle,codemodeexercice,...,Sans RDV,code dépar,Code Département,Nom Département,Nombre de médecins,Nombre de médecins pour 100 000 habitants,généralistes,spécialistes,dentistes,Pharm.
0,0.0,069204550,NaN,LAZZARESCHI,JULIE,92.0,Orthoptiste,C,Civil,L,...,0.0,06,06,Alpes-Maritimes,5095,461,173,288,123,123
1,8.0,10000030766,DR,NALLET,EMMANUEL,10.0,Médecin,C,Civil,L,...,0.0,971,971,Guadeloupe,1209,319,150,169,62,120
2,8.0,10000058080,DR,ARNAUD,LOUIS,40.0,Chirurgien-Dentiste,C,Civil,L,...,0.0,92,92,Hauts-de-Seine,6238,381,135,246,98,154
3,8.0,10000069749,DR,SALA,BERNARD,40.0,Chirurgien-Dentiste,C,Civil,L,...,0.0,75,75,Paris,18805,888,234,654,152,181
4,8.0,10000076546,DR,COHEN,THIERRY,40.0,Chirurgien-Dentiste,C,Civil,L,...,0.0,75,75,Paris,18805,888,234,654,152,181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,8.0,10100574408,DR,GALA,CHRISTOPHER,40.0,Chirurgien-Dentiste,C,Civil,L,...,0.0,06,06,Alpes-Maritimes,5095,461,173,288,123,123
329,8.0,10100582393,DR,ORSINI,DON PIERRE,40.0,Chirurgien-Dentiste,C,Civil,L,...,0.0,78,78,Yvelines,3891,267,108,159,63,89
330,8.0,10100636264,DR,CHAABAN,BASSEM,10.0,Médecin,C,Civil,L,...,0.0,75,75,Paris,18805,888,234,654,152,181
331,8.0,10100709970,DR,LYAZIDI,Souhad,10.0,Médecin,C,Civil,L,...,0.0,92,92,Hauts-de-Seine,6238,381,135,246,98,154


In [170]:
data_avec_concu.columns

Index(['typedidentifiantpp', 'identifiantpp', 'codecivilitédexercice',
       'nomdexercice', 'prénomdexercice', 'codeprofession',
       'libelléprofession', 'codecatégorieprofessionnelle',
       'libellécatégorieprofessionnelle', 'codemodeexercice',
       'libellémodeexercice', 'complémentdestinatairecoordstruc',
       'complémentpointgéographiquecoord', 'numérovoiecoordstructure',
       'indicerépétitionvoiecoordstructu', 'codetypedevoiecoordstructure',
       'libellétypedevoiecoordstructure', 'libellévoiecoordstructure',
       'mentiondistributioncoordstructur', 'bureaucedexcoordstructure',
       'codepostalcoordstructure', 'codecommunecoordstructure',
       'libellécommunecoordstructure', 'codepayscoordstructure',
       'libellépayscoordstructure', 'codedépartementstructure',
       'libellédépartementstructure', 'codesecteurdactivité',
       'libellésecteurdactivité', 'coderôle', 'libellérôle',
       'codegenreactivité', 'libellégenreactivité', 'Doctor Name', 'Specialt

In [171]:
data_final = data_avec_concu[['nomdexercice', 'prénomdexercice','RPPS','Specialty','card_0', 'card_3', 'card_4','Skills Processed','Nb Skills', 'moyens_paiement',
       'carte_vitale', 'secteur', 'Présentation', 'Langues parlées',
       'Diplômes nationaux et universitaires', 'Autres formations',
       'Expériences', 'Travaux et publications',
       'Prix et distinctions', 'site', 'Formations',
       'Diplômes nationaux et universitaires b', 'Nb caractères présentation',
       'Autres formations b', 'Nb langues', 'Expériences b', 'Carac_card_1',
       'Transport', 'Parking', 'Infos', 'Coordonnées', 'Sans RDV','Code Département', 'Nom Département',
       'Nombre de médecins', 'Nombre de médecins pour 100 000 habitants',
       'généralistes', 'spécialistes','dentistes']]
data_final.head()

,nomdexercice,prénomdexercice,RPPS,Specialty,card_0,card_3,card_4,Skills Processed,Nb Skills,moyens_paiement,...,Infos,Coordonnées,Sans RDV,Code Département,Nom Département,Nombre de médecins,Nombre de médecins pour 100 000 habitants,généralistes,spécialistes,dentistes
0,LAZZARESCHI,JULIE,069204550,Orthoptiste,"[Tarifs et remboursement, , , , Conventionné, ...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Bilan orthoptique, 39 €, Bilan orthop...","[Expertises et actes, Bilan neurovisuel, Bilan...",5.0,1.0,...,1.0,1.0,0.0,06,Alpes-Maritimes,5095,461,173,288,123
1,NALLET,EMMANUEL,10000030766,ORL - Chirurgien de la face et du cou,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,"[Expertises et actes, ORL et otologie, ORL péd...",4.0,0.0,...,0.0,1.0,0.0,971,Guadeloupe,1209,319,150,169,62
2,ARNAUD,LOUIS,10000058080,Chirurgien-dentiste,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,"[Expertises et actes, Apnée du sommeil, Chirur...",8.0,0.0,...,1.0,1.0,0.0,92,Hauts-de-Seine,6238,381,135,246,98
3,SALA,BERNARD,10000069749,Chirurgien-dentiste,"[Tarifs et remboursement, , Carte Vitale accep...",NaN,"[Tarifs, Consultation dentaire, 23 €, Detartra...","[Expertises et actes, Chirurgie buccale, Proth...",2.0,1.0,...,1.0,0.0,0.0,75,Paris,18805,888,234,654,152
4,COHEN,THIERRY,10000076546,Chirurgien-dentiste,"[Tarifs et remboursement, , , , Conventionné, ...","[Horaires et coordonnées, Horaires d'ouverture...",NaN,"[Expertises et actes, Blanchiment des dents, C...",11.0,1.0,...,1.0,1.0,0.0,75,Paris,18805,888,234,654,152
